네이버 블로그로 작성한 글을 cafe24 게시판에 옮기기 위해 네이버 블로그 글을 크롤링한 후 불필요한 태그를 제거하는 코드

In [1]:
import requests
import base64
import re
import json
import datetime 
import urllib
from time import time, sleep
from tqdm import tqdm_notebook
import ftplib
from bs4 import BeautifulSoup
from dateutil import parser
from selenium import webdriver


In [2]:
options = webdriver.ChromeOptions()  
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome('/Users/dooyeoung/Documents/webdriver/chromedriver', 
#                           chrome_options=options
)

 
blogurl = 'https://m.blog.naver.com/qweqweqwe/222078412158'
driver.get(blogurl)
contents = driver.find_elements_by_css_selector('.se-main-container .se-component')

In [27]:
html = ''
filter_li = []
for c in contents:
    class_list = c.get_attribute('class').replace('se-component','').replace('se-l-default','').strip()
    class_list = class_list.split(' ')

    if class_list[0] == 'se-text':
        html += '''
<div class="contents_text">
    {}
</div>
    '''.format('<br>'.join(c.text.split('\n')) )
    elif class_list[0] == 'se-sticker':
        html +='<br><br>'
    elif class_list[0] == 'se-image' :
        linkdata = eval(c.find_element_by_tag_name('a').get_attribute('data-linkdata'))
        linkdata['src'] = linkdata['src'] + '?type=w800'

        html += '''
<div class="contents_img"> 
    <img class="m-auto" src="{}">
</div>

    '''.format( linkdata['src']) 

    elif class_list[0] == 'se-horizontalLine':
        html += '<hr class="my-8"> '

    elif class_list[0] == 'se-imageStrip':
        itag_li = ''
        for img in  c.find_elements_by_tag_name('a'):
            linkdata = eval(img.get_attribute('data-linkdata'))
            linkdata['src'] = linkdata['src'] + '?type=w800'

            itag = '''
            <div class="flex-1"> 
                <div style=" height:300px;
                background-image : url({});
                background-repeat : no-repeat;
                background-size: cover;
                background-position: 50% 50%; ">
                </div>
            </div>
            '''.format( linkdata['src'])
            itag_li += itag


        html += '''
<div class="contents_img flex"> 
    {}
</div>
        '''.format(itag_li)


    elif class_list[0] == 'se-oglink':
        info = c.find_element_by_class_name('se-oglink-info-container') 
        ogthumb = c.find_elements_by_class_name('se-oglink-thumbnail')
        imgtag = ''
        if len(ogthumb) > 0 :
            ogthumb = ogthumb[0]
            imgsrc = ogthumb.find_element_by_css_selector('img').get_attribute('src')
            imgtag = '<img src="{}"></img>'.format(imgsrc) 

        html += '''
<div class="contents_oglink"> 
    {}
    {}
</div>
    '''.format(imgtag, info.get_attribute('outerHTML'))
        filter_li.append(c)
    elif class_list[0] == 'se-oembed':
        youtube = c.find_element_by_class_name('se-module-oembed')

    elif class_list[0] == 'se-quotation':
        html += '''
<div class="contents_text">
    <div class="text-2xl font-bold">{}</div>
    <div class="text-base text-grey">{}</div>
</div>'''.format(c.find_element_by_class_name('se-quote').text, c.find_element_by_class_name('se-cite').text)
    else:
        print(class_list)